# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-16 21:22:24] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-16 21:22:24] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-16 21:22:24] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-16 21:22:27] INFO server_args.py:1657: Attention backend not specified. Use fa3 backend by default.


[2026-01-16 21:22:27] INFO server_args.py:2556: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=76.42 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=72 avail_mem=76.29 GB):  25%|██▌       | 5/20 [00:00<00:01, 10.43it/s]

Capturing batches (bs=24 avail_mem=76.26 GB):  60%|██████    | 12/20 [00:00<00:00, 19.63it/s]

Capturing batches (bs=1 avail_mem=76.23 GB):  95%|█████████▌| 19/20 [00:01<00:00, 24.46it/s]

Capturing batches (bs=1 avail_mem=76.23 GB): 100%|██████████| 20/20 [00:01<00:00, 18.30it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Linlin. I'm 12 years old, I come from China. I'm a girl. I like singing and dancing. Now, I want to tell you something about China. China is a big country, and there are many beautiful places there. I like the south of China because it's very pretty. On the south side of the city of Xi'an, there is a big river called the Li River. The big river is very beautiful. There are many bridges and houses in the middle of the river. At the other side of the river, there is a big park. People like to go there to play. There
Prompt: The president of the United States is
Generated text:  running for a second term, and it's important that this happens within a certain period of time. The two years in which the president is eligible to run for another term are 2010 and 2014. If the president is only eligible to run for another term within 1 year after 2010 and 2014, and he needs a majority of the votes to win, what is the minimum number of votes the preside

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [insert a short description of your job or profession]. I'm always looking for new opportunities to grow and learn, and I'm always eager to learn more about the world around me. What do you do for a living? I'm always looking for new opportunities to grow and learn, and I'm always eager to learn more about the world around me. What do you enjoy doing? I enjoy reading, traveling, and learning new things. I

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. 

A. True
B. False
A. True

Paris is the capital city of France, and it is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. The city is also famous for its rich history, including the French Revolution and the French Revolution Monument. Paris is a major cultural and economic center in Europe and is home to many famous museums, theaters, and restaurants. The city is also known for its fashion industry, with many famous designers and boutiques located in the city. Overall, Paris is a fascinating and vibrant city that is a must-

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in several key areas, including:

1. Increased automation and robotics: AI is already being used in manufacturing, healthcare, and transportation, but it is likely to become even more prevalent in the future. Robots and AI-powered systems will be able to perform tasks that were previously done by humans, such as manufacturing, cleaning, and security.

2. Enhanced cognitive abilities: AI is likely to continue to improve its ability to process and analyze information, leading to new applications in fields such as education, healthcare, and finance.

3. Personalization and customization: AI will be able to learn from user data and provide



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I’m an artificial intelligence designed to assist people in a variety of ways. My purpose is to provide information, help with tasks, and even engage in conversation. I am not a person but a computer program, which is why I don’t have a physical body. I was created by a team of computer scientists, and I’m here to be a helpful assistant to those who use me.

I don’t have any personal interests or emotions, but I’m always ready to assist in answering questions or providing helpful information. I can talk to you for hours if you need me to, and I can even write reports or make

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. 

This statement captures the following key points:
- The capital of France, Paris
- Its status as the capital
- The number of countries, including France, in which Paris 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

insert

 name

 here

...

ability

 and

 accountability

 in

 AI

In [6]:
llm.shutdown()